In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import joblib
import nltk

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score, roc_auc_score
)

sns.set()

# Download stopwords on first run
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("english"))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df = pd.read_csv(r"d:\Hoax\Models\Fake Review - TF-IDF - Loistic Regression\data\deceptive-opinion.csv") # change filename

df.head()


deceptive   hotel  polarity       source  \
0  truthful  conrad  positive  TripAdvisor   
1  truthful   hyatt  positive  TripAdvisor   
2  truthful   hyatt  positive  TripAdvisor   
3  truthful    omni  positive  TripAdvisor   
4  truthful   hyatt  positive  TripAdvisor   

                                                text  
0  We stayed for a one night getaway with family ...  
1  Triple A rate with upgrade to view room was le...  
2  This comes a little late as I'm finally catchi...  
3  The Omni Chicago really delivers on all fronts...  
4  I asked for a high floor away from the elevato...

In [13]:
df.info()
df['deceptive'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   deceptive  1600 non-null   object
 1   hotel      1600 non-null   object
 2   polarity   1600 non-null   object
 3   source     1600 non-null   object
 4   text       1600 non-null   object
dtypes: object(5)
memory usage: 62.6+ KB


deceptive
truthful     800
deceptive    800
Name: count, dtype: int64

In [14]:
df['label'] = df['deceptive'].apply(lambda x: 1 if x == "deceptive" else 0)

In [15]:
X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [16]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_features=5000,    # good starting point
    ngram_range=(1,2)     # unigrams + bigrams improve accuracy
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [17]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=2000)

In [18]:
y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
cm

Accuracy: 0.8875

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.87      0.89       160
           1       0.87      0.91      0.89       160

    accuracy                           0.89       320
   macro avg       0.89      0.89      0.89       320
weighted avg       0.89      0.89      0.89       320



array([[139,  21],
       [ 15, 145]])

In [23]:
sample = ["This hotel was amazing, wonderful staff and great service!"]
sample_tfidf = vectorizer.transform(sample)
print(model.predict(sample_tfidf))
# 0 = Truthful
# 1 = Deceptive

[0]


In [21]:
sample = ["Absolutely the best hotel I’ve ever stayed in! The rooms were unbelievably luxurious, the staff treated us like royalty, and every single detail was perfect. I highly recommend this place to anyone visiting the city — trust me, you won’t find anything better!"]
sample_tfidf = vectorizer.transform(sample)
print(model.predict(sample_tfidf))
# 0 = Truthful
# 1 = Deceptive

[1]
